In [38]:
import numpy as np
import time

from zutils.ColorConsole import ColorCodes, clearOutput
from zutils.Moves import Moves
from zutils.BaseMap import BaseMap

The Warehouse class is for solving part #1

In [39]:
class Warehouse(BaseMap):
    __robot = '@'
    __wall = '#'
    __box = 'O'
    __empty = '.'

    def __init__(self, file):
        super().__init__()
        #set all board cells to zero
        self.defaultColor = ColorCodes.Grey
        # show cells without robots in Grey
        self.setColorMap({
            self.__wall: ColorCodes.Grey,
            self.__box: ColorCodes.Green,
            self.__empty: ColorCodes.Grey,
            self.__robot: ColorCodes.Red,
            '<': ColorCodes.Red,
            '>': ColorCodes.Red,
            'v': ColorCodes.Red,
            '^': ColorCodes.Red
        })
        self.moves = Moves()
        self.ignoreCharMap = { '.': ' '}
        self.currPosIndicator = self.__robot
        self.robotMoveCmds = ""
        self.wall = None
        self.readFile(file)


    def readFile(self, file):
        with open(file, 'r') as file:
            strBoard = ""
            for line in file:
                if line[0] == self.__wall:
                    strBoard += line
                elif line != "\n":
                    self.robotMoveCmds += line.replace("\n", '')
            self.createFromString(strBoard)
        self.wall = self.getIndicesOf('#')
        self.currPos = self.getIndicesOf('@')[0]
        self.board[self.currPos[0], self.currPos[1]] = self.__empty

    
    def moveObject(self, moveCmd, currObjIdx, currObj):
        nextObjIdx = self.moves.move(moveCmd, currObjIdx)
        nextObj = self.board[nextObjIdx[0], nextObjIdx[1]]
        match nextObj:
            case self.__wall:
                return (False, currObjIdx)      # You cannot Pass!
            case self.__box:
                move = self.moveObject(moveCmd, nextObjIdx, nextObj)
                if not move[0]:
                    return (False, currObjIdx)      # You cannot Pass!
                else:
                    if currObj == self.__box:
                        self.board[currObjIdx[0], currObjIdx[1]] = self.__empty
                        self.board[nextObjIdx[0], nextObjIdx[1]] = currObj
                    return (True, nextObjIdx)
            case _:
                # Empty space and any other char are ignored and treated as empty space
                #if current obj exchange box with 
                if currObj == self.__box:
                    self.board[currObjIdx[0], currObjIdx[1]] = self.__empty
                    self.board[nextObjIdx[0], nextObjIdx[1]] = currObj
                return (True, nextObjIdx)


    def robotMoveAll(self):
        for moveCmd in self.robotMoveCmds:
            clearOutput(wait=True)
            self.currPosIndicator = moveCmd
            move = self.moveObject(moveCmd, self.currPos, self.currPosIndicator)
            self.colorMap[self.currPosIndicator] = ColorCodes.Yellow if not move[0] else self.colorMap[self.__robot]

            #print(moveCmd, self.currPos, newPos)
            if move[0]:
                self.currPos = move[1]
            print(self)
            time.sleep(.03)


    def calcGPS(self):
        gps = 0
        boxes = self.getIndicesOf('O')
        for box in boxes:
            gps += (box[0] * 100 + box[1])
        return gps   

Wide warehouse is for solving part #2

In [40]:
class WideWarehouse(BaseMap):
    __robot = '@'
    __wall = '#'
    __box = 'O'
    __empty = '.'
    __boxL = '['
    __boxR = ']'

    def __init__(self, file):
        super().__init__()
        #set all board cells to zero
        self.defaultColor = ColorCodes.Grey
        # show cells without robots in Grey
        self.setColorMap({
            self.__wall: ColorCodes.Grey,
            self.__box: ColorCodes.Green,
            self.__boxL: ColorCodes.Green,
            self.__boxR: ColorCodes.Blue,
            self.__empty: ColorCodes.Grey,
            self.__robot: ColorCodes.Red,
            '<': ColorCodes.Red,
            '>': ColorCodes.Red,
            'v': ColorCodes.Red,
            '^': ColorCodes.Red
        })
        self.moves = Moves()
        self.ignoreCharMap = { '.': ' '}
        self.currPosIndicator = self.__robot
        self.robotMoveCmds = ""
        self.wall = None
        self.readFile(file)


    def readFile(self, file):
        lol = []
        with open(file, 'r') as file:
            for line in file:
                if line[0] == self.__wall:
                    l = []
                    line = line.replace("\n", '')
                    for char in line:
                        match char:
                            case self.__robot:
                                l += self.__robot, self.__empty
                            case self.__wall | self.__empty:
                                l += [char] * 2
                            case self.__box:
                                l += self.__boxL, self.__boxR
                    lol += [l]
                elif line != "\n":
                    self.robotMoveCmds += line.replace("\n", '')
            self.createFromList(lol)
        self.wall = self.getIndicesOf('#')
        self.currPos = self.getIndicesOf('@')[0]
        self.board[self.currPos[0], self.currPos[1]] = self.__empty


    def addIfNotInCache(self, key, value, cache):
        if key not in cache:
            cache[key] = value
    

    # future moves cache is a dictionary key=(moveCmd, currPos) val=(movedFlg, newObjIdx, currObjIdx, currObj, futureLevel)
    # pending moves is a set of pending moves (moveCmd, tuple(currObjIdx))
    def moveObject(self, moveCmd, currObjIdx, currObj, futureLevel, futureMovesCache, pendingMoves):
        currObjKey = (moveCmd, tuple(currObjIdx))
        pendingMoves.add(currObjKey)
        nextObjIdx = self.moves.move(moveCmd, currObjIdx)
        nextObj = str(self.board[nextObjIdx[0], nextObjIdx[1]])
        if currObjKey in futureMovesCache:
            retVal = futureMovesCache[currObjKey]
        else:
            match nextObj:
                case self.__wall:
                    retVal = (False, currObjIdx, currObjIdx, currObj, futureLevel)   # You cannot Pass!
                case self.__boxL | self.__boxR:
                    secHalfIdx = self.moves.move('>' if nextObj == self.__boxL else '<', nextObjIdx)
                    secHalfObj = self.__boxR if nextObj == self.__boxL else self.__boxL
                    keyFirstHalf = (moveCmd, tuple(nextObjIdx)) 
                    keySecondHalf = (moveCmd, tuple(secHalfIdx))
                    firstHalfMove =  self.moveObject(moveCmd, nextObjIdx, nextObj, futureLevel + 1, futureMovesCache, pendingMoves) if keyFirstHalf not in futureMovesCache else futureMovesCache[keyFirstHalf] 
                    canMove = firstHalfMove[0]
                    if keySecondHalf not in pendingMoves:
                        secHalfMove =  self.moveObject(moveCmd, secHalfIdx, secHalfObj, futureLevel + 1, futureMovesCache, pendingMoves) if keySecondHalf not in futureMovesCache else futureMovesCache[keySecondHalf] 
                        canMove = (canMove and secHalfMove[0])
                    if not canMove:
                        retVal = (False, currObjIdx, currObjIdx, currObj, futureLevel)   # You cannot Pass!
                    else:
                        retVal = (True, nextObjIdx, currObjIdx, currObj, futureLevel)

                case _:
                    # Empty space and any other char are ignored and treated as empty space, can move there
                    retVal = (True, nextObjIdx, currObjIdx, currObj, futureLevel)
                    
        self.addIfNotInCache(currObjKey, retVal, futureMovesCache)
        pendingMoves.remove(currObjKey)
        #print(futureMovesCache)
        #time.sleep(.1)
        return retVal


    def robotMoveAll(self):
        progress = 1
        for moveCmd in self.robotMoveCmds:
            clearOutput(wait=True)
            self.currPosIndicator = moveCmd
            # future moves is a dictionary key=(moveCmd, currPos) val=(movedFlg, newObjIdx, currObjIdx, currObj, futureLevel)
            futureMovesCache = {}
            pendingMoves = set()
            move = self.moveObject(moveCmd, self.currPos, self.currPosIndicator, 0, futureMovesCache, pendingMoves)
            futureMoves = list(sublist for sublist in futureMovesCache.values())
            movedEach = list(sublist[0] for sublist in futureMovesCache.values())
            if all(movedEach):
                #print("moved each")
                sortedFutureMoves = sorted(futureMoves, key=lambda x: x[4], reverse=True)
                #print(sortedFutureMoves)
                for item in sortedFutureMoves:
                    nextObjIdx = item[1]
                    currObjIdx = item[2]
                    currObj = item[3]
                    self.board[currObjIdx[0], currObjIdx[1]] = self.__empty
                    self.board[nextObjIdx[0], nextObjIdx[1]] = currObj
            self.colorMap[self.currPosIndicator] = ColorCodes.Yellow if not move[0] else self.colorMap[self.__robot]

            if move[0]:
                self.currPos = move[1]
            print("Progress: {0} %".format(int(progress/len(self.robotMoveCmds)*100)))
            progress += 1
            print(self)
            time.sleep(.03)


    def calcGPS(self):
        gps = 0
        boxes = self.getIndicesOf('[')
        for box in boxes:
            gps += (box[0] * 100 + box[1])
        return gps   

In [41]:
fileName = 'input/Day15-mini.txt'
cost = [0,0]

warehouse = Warehouse(fileName)
warehouse.robotMoveAll()
cost[0] = warehouse.calcGPS()

wideWarehouse = WideWarehouse(fileName)
wideWarehouse.robotMoveAll()
cost[1] = wideWarehouse.calcGPS()

print("Part 1 - safety factor: {0}".format(cost[0]))
print("Part 2 - safety factor: {0}".format(cost[1]))


Progress: 100 %
                    1 1 1 1 1 1 1 1 1 1
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9
________________________________________
# # # # # # # # # # # # # # # # # # # # |0
# # [ ]               [ ]   [ ] [ ] # # |1
# # [ ]                       [ ]   # # |2
# # [ ]                 [ ] [ ] [ ] # # |3
# # [ ]             [ ]         [ ] # # |4
# #     # #             [ ]         # # |5
# #     [ ]                         # # |6
# #     ^             [ ]   [ ] [ ] # # |7
# #             [ ] [ ]     [ ]     # # |8
# # # # # # # # # # # # # # # # # # # # |9

Part 1 - safety factor: 10092
Part 2 - safety factor: 9021


In [ ]:
#playing with iterators
from itertools import product
from itertools import combinations
from itertools import permutations

rng = range(1,5)

cartesian = product(rng, repeat=2)
combs = combinations(rng, r=2)
perms = permutations(rng, r=2)

for item in cartesian:
    print(item)

print("Combs")
for item in combs:
    print(item)

print("Perms")
for item in perms:
    print(item)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
Combs
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
Perms
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
